<a href="https://colab.research.google.com/github/SARITHASAKTHIVEL/Security-dashboard/blob/main/ruleengine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.4 MB/s eta 0:00:00


In [15]:
# Open the file and print the relevant lines
with open('eq-store.csv', 'r') as file:
    for i, line in enumerate(file):
        if 30 <= i <= 34:  # Print around line 32
            print(f"Line {i+1}: {line}")

Line 31: financial,[a-zA-Z0-9._%+-]+@idfcbank,UPI ID,10

Line 32: financial,"[a-zA-Z0-9._%+-]+@waicici, @icici",UPI ID,10

Line 33: financial,[a-zA-Z0-9._%+-]+@waaxis,UPI ID,10

Line 34: financial,[a-zA-Z0-9._%+-]+@wahdfcbank,UPI ID,10

Line 35: financial,[a-zA-Z0-9._%+-]+@wasbi,UPI ID,10



In [14]:
pip install chardet


In [15]:
import chardet

# Open the file in binary mode and detect encoding
with open('eq-store.csv', 'rb') as f:
    result = chardet.detect(f.read())

print(result['encoding'])
# Now use the detected encoding to read the file
df = pd.read_csv('rule-engine.csv', encoding=result['encoding'])


UTF-8-SIG


In [16]:
import pandas as pd
df=pd.read_csv('eq-store.csv')
df.head(10)

,Category,Field,type of data,Label
0,financial,[a-zA-Z0-9._%+-]+@apl,UPI ID,10
1,financial,[a-zA-Z0-9._%+-]+@yapl,UPI ID,10
2,financial,[a-zA-Z0-9._%+-]+@rapl,UPI ID,10
3,financial,[a-zA-Z0-9._%+-]+@abfspay,UPI ID,10
4,financial,[a-zA-Z0-9._%+-]+@abfspay,UPI ID,10
5,financial,[a-zA-Z0-9._%+-]+@axisb,UPI ID,10
6,financial,[a-zA-Z0-9._%+-]+@yescred,UPI ID,10
7,financial,[a-zA-Z0-9._%+-]+@idfcbank,UPI ID,10
8,financial,[a-zA-Z0-9._%+-]+@fkaxis,UPI ID,10
9,financial,[a-zA-Z0-9._%+-]+@icici,UPI ID,10


In [3]:
import json
def convert_gpt(dataset):
  fine_tunning_data=[]
  for _, row in dataset.iterrows():
    # Ensure both 'Category' and 'Label' are converted to strings
    json_response='{"Category":'+ str(row['Category']) +'", "Label":'+ str(row['Label'])+'}'
    fine_tunning_data.append({
        "messages":[{"role":"user","content":row['Field']},
                    {"role":"assistant","content":json.dumps(json_response)}
                    ]
    })
  return fine_tunning_data

dataset=pd.read_csv('rule-engine.csv')
convert_data=convert_gpt(dataset)
convert_data[0]['messages']
json.loads(convert_data[0:10][0]['messages'][-1]['content'])

'{"Category":financial", "Label":10}'

In [4]:
!pip install datasets

from datasets import Dataset


df = pd.read_csv('eq-store.csv')

# Combine category and label into a single output
df['target'] = df['Category'].astype(str) + ' - ' + df['Label'].astype(str)

# Prepare data for fine-tuning (example format)
fine_tuning_data = []
for _, row in df.iterrows():
  fine_tuning_data.append({'text': row['Field'], 'label': row['target']})

print(fine_tuning_data)

[{'text': '[a-zA-Z0-9._%+-]+@apl', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@yapl', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@rapl', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@abfspay', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@abfspay', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@axisb', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@yescred', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@idfcbank', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@fkaxis', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@icici', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@okaxis', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@okhdfcbank', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@okicici', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@oksbi', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]+@yesg', 'label': 'financial - 10'}, {'text': '[a-zA-Z0-9._%+-]

In [5]:
!pip install transformers datasets
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
from datasets import Dataset


In [6]:
model_name = "distilbert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(df['target'].unique()))
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Convert data to Hugging Face Dataset


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
dataset = Dataset.from_list([{k: str(v) for k, v in x.items()} for x in fine_tuning_data])

In [8]:
label2id = {label: i for i, label in enumerate(df['target'].unique())}
id2label = {i: label for label, i in label2id.items()}


In [9]:
def map_labels(example):
  example['label'] = label2id[example['label']]
  return example

# Apply the label mapping to the dataset
dataset = dataset.map(map_labels)

tokenizer = AutoTokenizer.from_pretrained(model_name)


Map:   0%|          | 0/574 [00:00<?, ? examples/s]

In [10]:
def preprocess_function(examples):
  return tokenizer(examples['text'], padding="max_length", truncation=True)


encoded_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/574 [00:00<?, ? examples/s]

In [18]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,  # Enable loading the best model
    metric_for_best_model="eval_loss", # Use validation loss for selection

)


In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset, #Use encoded_dataset for training
    eval_dataset=encoded_dataset, # Use encoded_dataset for evaluation (ideally, use a separate validation set)
)

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
